<a href="https://colab.research.google.com/github/Jonchyk/Datamgmt/blob/main/PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set Up

In [25]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
# Install tabula-py using pip
!pip install tabula-py
# Import the correct module
from tabula import read_pdf
from statistics import * #stats'


!pip install pyreadstat


import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

import urllib  #weird, guess need to have os and pandas imported for this to work  %TODO/LATER ditch it, its weird anyway, just use wget/curl

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg
##https://www.marktechpost.com/2023/10/20/6-magic-commands-for-jupyter-notebooks-in-python-data-science/
#%%latex
#%ai
#%run
#%writefile
#%history -n

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

#sometimes have to install library which you get from https://pypi.org/
#!pip install geopandas

# Background
For PS1, I will be running several datasets that I will be cleaning and hopefully using to get to my Capstone research question. That research question is simple:
"Can the disparity in national scholarship test scores (a test taken by teenagers), found between language of instructions and regions, also be found as early as Grade 2 in the same regions and for the same languages?

# Dataset 1: Foundational Learning Asessment -https://mics.unicef.org/news/just-released-kyrgyzstan-2023-mics-survey-findings-snapshots-and-datasets
an SPSS file found from UNICEF's Multi Cluster study in Kyrgystan, 2023 -
The plan is to understand this dataset a bit better, clean it up, and make it something potentially useful. I think i'll keep it as a second assessment of foundational literacy and math, as a seperate measure to compare against the EGRA

# UN Global Data Lab  - https://globaldatalab.org/areadata/profiles/KGZt/
The second dataset is population in millions in Kyrgyzstan. I'll run it, clean it, and calculate the percentage change per region year over year.
purpose of this dataset is to understand demographic changes within Kyrgyzstan, leading up to the MCIS
i'll also calculate a compound growth rate, to get a grasp on each region's change leading up to 2023.

#Bureau of Statistics, Kyrgyzstan - Per Capita Income: https://stat.gov.kg/en/opendata/category/122/
The third Dataset is going to just look at the average income per region. Nothing fancy, but the json file seems all messed up for some reason I can't understand.

#Early Grade Reading Assessment: received with permission from employer, RTI
The fourth dataset is from the EGRA. I'm going to clean this up so it fits my needs for later in this class and visualization.

#Disaster Incidence by Region - Kyrgyzstan https://data.opendata.am/dataset/inform-subnational-risk-index-2021-caucasus-and-central-asia
My fifth dataset comes from the INFORM subnational risk dataset, description here "INFORM identifies areas at a high risk of humanitarian crisis that are more likely to require international assistance. The INFORM model is based on risk concepts published in scientific literature and envisages three dimensions of risk: Hazards & Exposure, Vulnerability and Lack of Coping Capacity. The INFORM model is split into different levels to provide a quick overview of the underlying factors leading to humanitarian risk. The INFORM subnational model for Caucasus and Central Asia is developed at the first administrative level (corresponding to the provinces/oblasts/regions and few independent cities) of the eight countries in South Caucasus and Central Asia."

I will review this data to see what I can analyze and use for future PS to also see how geographic incidents/impact could potentially impact educational outcomes - this is something literature review notes as a significant and compounding issue in Kyrgyzstan.



# Data Set 1 - UNICEF Multi Cluster Analysis Dataset (MICS)

In [ ]:
#getting the first file
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb' -O mics.sav


--2025-02-08 18:06:21--  https://docs.google.com/uc?export=download&id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb
Resolving docs.google.com (docs.google.com)... 108.177.96.100, 108.177.96.139, 108.177.96.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.96.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download [following]
--2025-02-08 18:06:21--  https://drive.usercontent.google.com/download?id=1__OaL9WNTqnVrOl2VuZQJfc9LIdYKlXb&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.218.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1807033 (1.7M) [application/octet-stream]
Saving to: ‘mics.sav’

mics.sav            100%[===================>]   1.72M  --.-KB/s    in 0.

In [ ]:
#read spss file
ed = pd.read_spss("mics.sav")
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns', None)  # No limit on columns
#let's take a peek at this data set.
ed.head(10)
#Let's specifically look at our geographic breakdown, since this is important
ed["stratum"].value_counts()

,HH1,HH2,LN,FS1,FS2,FS3,FSINT,FS4,FS5,FS6,FS7D,FS7M,FS7Y,FS9,FS10,FS17,FS8H,FS8M,FS11H,FS11M,FS12,FS13,FS14,FS15,FSHINT,FSFIN,FS15A,FS15C,FS15D,CB2M,CB2Y,CB3,CB4,CB5A,CB5B,CB6,CB7,CB8A,CB8B,CB9,CB10A,CB10B,CL1A,CL1B,CL1C,CL1X,CL3,CL4,CL5,CL6A,CL6B,CL6C,CL6D,CL6E,CL6X,CL7,CL8,CL9,CL10,CL11A,CL11B,CL11C,CL11D,CL11E,CL11F,CL11X,CL13,FCD2A,FCD2B,FCD2C,FCD2D,FCD2E,FCD2F,FCD2G,FCD2H,FCD2I,FCD2J,FCD2K,FCD3,FCD4,FCD5,FCF1,FCF2,FCF3,FCF6,FCF8,FCF10,FCF11,FCF12,FCF13,FCF14,FCF15,FCF16,FCF17,FCF18,FCF19,FCF20,FCF21,FCF22,FCF23,FCF24,FCF25,FCF26,PR3,PR5,PR6,PR7,PR8,PR9A,PR9B,PR10,PR11A,PR11B,PR12A,PR12B,PR12C,PR12X,PR13,PR15,FL1,FL2H,FL2M,FL3,FL4A,FL4B,FL4C,FL6A,FL6B,FL7,FL9,FL10,FL10C,FL13,FL14,FL15,FL16,FL17,FL18,FL19INTRO,FL19W1,FL19W2,FL19W3,FL19W4,FL19W5,FL19W6,FL19W7,FL19W8,FL19W9,FL19W10,FL19W11,FL19W12,FL19W13,FL19W14,FL19W15,FL19W16,FL19W17,FL19W18,FL19W19,FL19W20,FL19W21,FL19W22,FL19W23,FL19W24,FL19W25,FL19W26,FL19W27,FL19W28,FL19W29,FL19W30,FL19W31,FL19W32,FL19W33,FL19W34,FL19W35,FL19W36,FL19W37,FL19W38,FL19W39,FL19W40,FL19W41,FL19W42,FL19W43,FL19W44,FL19W45,FL19W46,FL19W47,FL19W48,FL19W49,FL19W50,FL19W51,FL19W52,FL19W53,FL19W54,FL19W55,FL19W56,FL19W57,FL19W58,FL19W59,FL19W60,FL19W61,FL19W62,FL19W63,FL19W64,FL19W65,FL19W66,FL19W67,FL19W68,FL19W69,FL19W70,FL19W71,FL19W72,FL19END,FL20A,FL20B,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE,FL21D,FL21G,FL21H,FL21I,FL21J,FL21K,FL21L,FL21OINTRO,FL21OW1,FL21OW2,FL21OW3,FL21OW4,FL21OW5,FL21OW6,FL21OW7,FL21OW8,FL21OW9,FL21OW10,FL21OW11,FL21OW12,FL21OW13,FL21OW14,FL21OW15,FL21OW16,FL21OW17,FL21OW18,FL21OW19,FL21OW20,FL21OW21,FL21OW22,FL21OW23,FL21OW24,FL21OW25,FL21OW26,FL21OW27,FL21OW28,FL21OW29,FL21OW30,FL21OW31,FL21OW32,FL21OW33,FL21OW34,FL21OW35,FL21OW36,FL21OW37,FL21OW38,FL21OW39,FL21OW40,FL21OW41,FL21OW42,FL21OW43,FL21OW44,FL21OW45,FL21OW46,FL21OW47,FL21OW48,FL21OW49,FL21OW50,FL21OW51,FL21OW52,FL21OW53,FL21OW54,FL21OEND,FL21PA,FL21PB,FL22A,FL22B,FL22C,FL22D,FL22E,FL23A,FL23B,FL23C,FL23D,FL23E,FL23F,FL24A,FL24B,FL24C,FL24D,FL24E,FL25A,FL25B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,ST1LANG,ST1WNUM,ST2LANG,ST2WNUM,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,FSAGE,FSDOI,FSDOB,fshweight,fsweight,melevel,fselevel,schage,fsdisability,ethnicity,caretakerdis,wscore,windex5,windex10,wscoreu,windex5u,windex10u,wscorer,windex5r,windex10r,PSU,stratum
0,1.0,8.0,3.0,1.0,8.0,3.0,35.0,2.0,35.0,30.0,22.0,11.0,2023.0,"NO, FIRST INTERVIEW",YES,COMPLETED,17.0,30.0,17.0,53.0,KYRGYZ,KYRGYZ,KYRGYZ,"NO, NOT USED",35.0,SAVE QUESTIONNAIRE AND FINISH,NO,NO,NaN,MAY,2011.0,12.0,YES,BASIC SECONDARY,GRADE 6,NO,YES,BASIC SECONDARY,GRADE 6,YES,BASIC SECONDARY,GRADE 5,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,NO,NaN,YES,NO,YES,NO,NO,NO,YES,5.0,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO DIFFICULTY,NO DIFFICULTY,NaN,NaN,NaN,NaN,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,NO DIFFICULTY,A FEW TIMES A YEAR,NEVER,TEN OR MORE BOOKS,YES,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NaN,"YES, PERMISSION IS GIVEN",17.0,30.0,YES,You are not alone with the child unless he/she...,You have engaged the child in conversation and...,"The child is sat comfortably, able to use the ...",YES,NO,RUSSIAN,RUSSIAN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONTINUE,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,CORRECT,NOT REACHED,CONTINUE,71.0,0.0,CORRECT(TWO),CORRECT(FLOWERS),CORRECT(BECAUSE HE FELL),CORRE

,count
stratum,
Naryn - rural,480
Talas - rural,443
Osh city - urban,430
Osh - rural,387
Chui - rural,360
Batken - rural,327
Bishkek city - urban,308
Jala-Abad - rural,296
Issyk-kul - rural,284


In [ ]:
#let's get a full list of columns. What a mess!
print(list(ed.columns))

['HH1', 'HH2', 'LN', 'FS1', 'FS2', 'FS3', 'FSINT', 'FS4', 'FS5', 'FS6', 'FS7D', 'FS7M', 'FS7Y', 'FS9', 'FS10', 'FS17', 'FS8H', 'FS8M', 'FS11H', 'FS11M', 'FS12', 'FS13', 'FS14', 'FS15', 'FSHINT', 'FSFIN', 'FS15A', 'FS15C', 'FS15D', 'CB2M', 'CB2Y', 'CB3', 'CB4', 'CB5A', 'CB5B', 'CB6', 'CB7', 'CB8A', 'CB8B', 'CB9', 'CB10A', 'CB10B', 'CL1A', 'CL1B', 'CL1C', 'CL1X', 'CL3', 'CL4', 'CL5', 'CL6A', 'CL6B', 'CL6C', 'CL6D', 'CL6E', 'CL6X', 'CL7', 'CL8', 'CL9', 'CL10', 'CL11A', 'CL11B', 'CL11C', 'CL11D', 'CL11E', 'CL11F', 'CL11X', 'CL13', 'FCD2A', 'FCD2B', 'FCD2C', 'FCD2D', 'FCD2E', 'FCD2F', 'FCD2G', 'FCD2H', 'FCD2I', 'FCD2J', 'FCD2K', 'FCD3', 'FCD4', 'FCD5', 'FCF1', 'FCF2', 'FCF3', 'FCF6', 'FCF8', 'FCF10', 'FCF11', 'FCF12', 'FCF13', 'FCF14', 'FCF15', 'FCF16', 'FCF17', 'FCF18', 'FCF19', 'FCF20', 'FCF21', 'FCF22', 'FCF23', 'FCF24', 'FCF25', 'FCF26', 'PR3', 'PR5', 'PR6', 'PR7', 'PR8', 'PR9A', 'PR9B', 'PR10', 'PR11A', 'PR11B', 'PR12A', 'PR12B', 'PR12C', 'PR12X', 'PR13', 'PR15', 'FL1', 'FL2H', 'FL2M',

In [ ]:
# I want to create a column that is the percentage of words correct, taken from the test, for each student.
# to do so I know I need to convert FL20A and FL20B to numeric from categorical data, to get around the errors; then let's get the percentage of words attempted vs incorrect words for a "reading" score.
ed['FL20A'] = pd.to_numeric(ed['FL20A'], errors='coerce')
ed['FL20B'] = pd.to_numeric(ed['FL20B'], errors='coerce')
#Now that i've resolved the number vs categorical error, let's create a new column using some basic math.
ed['wrdprctcorrect'] = ((ed['FL20A'] - ed['FL20B']) / ed['FL20A']) * 100

# Oops. The above equation will come out all sorts of wrong if I don't ensure that someone with a perfect score isn't dividing by 0. values for cases where FL20B is 0 (all correct attempts)
ed.loc[ed['FL20B'] == 0, 'wrdprctcorrect'] = 100



In [ ]:
#Now I need to get something else...what are the values in these five columns? FL21BA	FL21BB	FL21BC	FL21BD	FL21BE. These are the reading comprehension questions.
ed[['FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE']].apply(lambda col: col.value_counts())



,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE
CORRECT (BECAUSE THE FARMER GAVE HIM MANY FLOWERS / BECAUSE HE HAD FLOWERS TO GIVE TO HIS MOTHER),NaN,NaN,NaN,NaN,1772.0
CORRECT(BECAUSE HE FELL),NaN,NaN,1814.0,NaN,NaN
CORRECT(FLOWERS),NaN,1898.0,NaN,NaN,NaN
CORRECT(NEAR A KARAGACH TREE),NaN,NaN,NaN,1551.0,NaN
CORRECT(TWO),1899.0,NaN,NaN,NaN,NaN
INCORRECT,35.0,37.0,96.0,348.0,144.0
NO RESPONSE,1.0,NaN,5.0,3.0,3.0
"NO RESPONSE/ SAYS ""I DONT KNOW""",8.0,8.0,28.0,41.0,24.0


In [ ]:
# Replace NaN with 0 first, then apply the conditions
ed['FL21BA'] = np.where(ed['FL21BA'] == "CORRECT(TWO)", 1, 0)
ed['FL21BB'] = np.where(ed['FL21BB'] == "CORRECT(FLOWERS)", 1, 0)
ed['FL21BC'] = np.where(ed['FL21BC'] == "CORRECT(BECAUSE HE FELL)", 1, 0)
ed['FL21BD'] = np.where(ed['FL21BD'] == "CORRECT(NEAR A KARAGACH TREE)", 1, 0)
ed['FL21BE'] = np.where(ed['FL21BE'] == "CORRECT(BECAUSE THE FARMER GAVE HIM MANY FLOWERS / BECAUSE HE HAD FLOWERS TO GIVE TO HIS MOTHER)", 1, 0)

# Handle NaN values after applying the conditions
ed[['FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE']] = ed[['FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE']].fillna(0)





In [ ]:
#checking did this work? FINALLY!
ed[['FL21BA', 'FL21BB', 'FL21BC', 'FL21BD', 'FL21BE']]

,FL21BA,FL21BB,FL21BC,FL21BD,FL21BE
0,1,1,1,1,0
1,0,0,0,0,0
2,1,1,1,1,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
3932,0,0,0,0,0
3933,0,0,0,0,0
3934,0,0,0,0,0
3935,0,0,0,0,0


In [ ]:
#finally, I need to create a "reading comprhension" score , which will be the sum of FL21BA, BB, BC, BD, and BE
ed['readingcomp'] = ed['FL21BA'] + ed['FL21BB'] + ed['FL21BC'] + ed['FL21BD'] + ed['FL21BE']

In [ ]:
#ok. Now that i've done this. I need to identify all of the "columns" that need to be kept. it's a lot...
#Let's drop all non necessary columns here, keeping language/geographic/ethnicity, grade, gender, age,  and the  foundational assessment questionaire
ed = ed.loc[:, ed.columns.isin(['FS10', 'FS12', 'FS13', 'FS14', 'CB3', 'CB5B' , 'wrdprctcorrect', 'readingcomp', 'FL3' , 'FL7','FL9', 'FL20A', 'FL20B', 'FL23A', 'FL23B', 'FL25C', 'FL25D', 'FL25E', 'FL26', 'FL26A', 'FL26B', 'FL26C', 'FL26D', 'FL26E', 'FL27A', 'FL27B', 'FL27C', 'FL27D', 'FL27E', 'FL28', 'HH4', 'HH6', 'HH7', 'HH52', 'HL4', 'ED5A', 'ED5B', 'ethnicity','stratum'])]

In [ ]:
ed.head(1)

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,stratum,wrdprctcorrect,readingcomp
0,YES,KYRGYZ,KYRGYZ,KYRGYZ,12.0,GRADE 6,YES,RUSSIAN,RUSSIAN,71.0,0.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 2,Russian,Issyk-kul - urban,100.0,4


In [ ]:
#let's get this down to 7 to 10 years old, since the data collected is for ages 7-14, but is a G2 foundational test. Wow!
ed_age= ed[(ed["CB3"] > 6) & (ed["CB3"] < 11)]
#how many respondents do we have now?
ed_age["stratum"].value_counts()

,count
stratum,
Naryn - rural,152
Osh city - urban,143
Osh - rural,138
Talas - rural,132
Chui - rural,128
Batken - rural,106
Issyk-kul - rural,100
Bishkek city - urban,98
Jala-Abad - rural,97


In [ ]:
#further, let's see how many of these children consented to taking the test.
ed_age["FL3"].value_counts()

,count
FL3,
YES,1219
NO / NOT ASKED,48


In [ ]:
#1219 said yes! Of those 1219, how many do we have in each "stratum" now?
ed_age[["stratum", "FL3"]].value_counts(normalize=False)
#ok! This looks like it's enough data to work with.  While some of the urban smaples are a bit low, every sample per region is >30!


,,count
stratum,FL3,
Naryn - rural,YES,149
Osh city - urban,YES,130
Talas - rural,YES,128
Chui - rural,YES,124
Osh - rural,YES,114
Batken - rural,YES,100
Issyk-kul - rural,YES,100
Bishkek city - urban,YES,80
Jala-Abad - rural,YES,78


In [ ]:
#what are the counts for each grade.
ed_age["CB5B"].value_counts( normalize=False)

,count
CB5B,
GRADE 2,349
GRADE 3,331
GRADE 4,304
GRADE 1,217
GRADE 5,111
GRADE 6,7
GRADE 10,0
GRADE 11,0
GRADE 7,0


In [ ]:
#I want to filter out everyone that isn't in Grade 2 , since this is a grade 2 foundational test
grades = ed_age[ed_age["CB5B"].isin(["GRADE 2"])]
#let's run this and see the count by stratum. Is this enough to be representative? Maybe ok if done at regional (non urban/rural level)
grades["stratum"].value_counts( normalize=False)


,count
stratum,
Osh - rural,45
Chui - rural,38
Osh city - urban,38
Naryn - rural,32
Jala-Abad - rural,29
Issyk-kul - rural,27
Talas - rural,26
Batken - rural,25
Bishkek city - urban,19


In [ ]:
#Another question remains; the questionaire asks children what language they speak at home, versus hear at school (FL7, FL9). How does those counts come about?
pd.crosstab(grades['FL7'], grades['FL9'])
#by basis of ethnicity compared to language of instruction
pd.crosstab(grades['ethnicity'], grades['FL9'])
#by basis of ethnicity compared to language at home
pd.crosstab(grades['ethnicity'], grades['FL7'])
#Looks like its most common to speak kyrgyz at home and at school.


FL9,KYRGYZ,RUSSIAN,TAJIK,UZBEK
FL7,,,,
16.0,0,1,0,0
KYRGYZ,188,52,0,1
RUSSIAN,1,45,0,0
TAJIK,0,0,1,0
UZBEK,3,13,0,14


FL9,KYRGYZ,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,
Kyrgyz,186,70,0,1
Other ethnicity,2,13,1,1
Russian,0,10,0,0
Uzbek,4,18,0,13


FL7,16.0,KYRGYZ,RUSSIAN,TAJIK,UZBEK
ethnicity,,,,,
Kyrgyz,0,234,21,0,2
Other ethnicity,1,4,9,1,2
Russian,0,0,10,0,0
Uzbek,0,3,6,0,26


In [ ]:
#Let's rename certain columns, not all of them, but a few critical pieces - still working on this
grades.rename(columns={'FS10': 'consent', 'FS12': 'language_questionaire', 'CB3':'Age', 'CB5B':'Grade', 'FS13': 'language_interview', 'FS14': 'native_language', 'CB5': 'highest_grade', 'HL4' : 'gender', 'HH6' : "urbrur", 'HH7' : 'region'}, inplace=False)

,consent,language_questionaire,language_interview,native_language,Age,Grade,FL3,FL7,FL9,FL20A,FL20B,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,HH4,urbrur,region,HH52,gender,ED5A,ED5B,ethnicity,stratum,wrdprctcorrect,readingcomp
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,2.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,2.0,FEMALE,HIGHER,GRADE 5,Other ethnicity,Issyk-kul - urban,97.183099,4
46,YES,RUSSIAN,RUSSIAN,RUSSIAN,7.0,GRADE 2,YES,RUSSIAN,RUSSIAN,NaN,NaN,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,INCORRECT,NaN,CONTINUE,CORRECT (8),INCORRECT,INCORRECT,INCORRECT,CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,3.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Russian,Issyk-kul - urban,NaN,0
51,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,3.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,INCORRECT,CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,3.0,MALE,PRIMARY,GRADE 4,Other ethnicity,Issyk-kul - urban,95.774648,4
69,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,YES,KYRGYZ,KYRGYZ,59.0,1.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,2.0,MALE,COMPLETE SECONDARY,GRADE 11,Kyrgyz,Issyk-kul - urban,98.305085,4
79,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,RUSSIAN,RUSSIAN,71.0,0.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,1.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,Issyk-kul - urban,100.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3910,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,YES,KYRGYZ,KYRGYZ,15.0,44.0,NO ATTEMPT,CORRECT( 12 ),CORRECT (10),INCORRECT,INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),INCORRECT,NO ATTEMPT,INCORRECT,COMPLETED,90.0,URBAN,Osh c,2.0,MALE,BASIC SECONDARY,GRADE 9,Kyrgyz,Osh city - urban,-193.333333,0
3912,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,KYRGYZ,59.0,0.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),INCORRECT,INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),INCORRECT,INCORRECT,COMPLETED,90.0,URBAN,Osh c,2.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,Osh city - urban,100.000000,4
3920,YES,KYRGYZ,KYRGYZ,KYRGYZ,8.0,GRADE 2,NO / NOT ASKED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHILD REFUSED,90.0,URBAN,Osh c,1.0,MALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,Osh city - urban,NaN,0
3921,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,0.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),CORRECT (36),CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,90.0,URBAN,Osh c,2.0,FEMALE,PROFESSIONAL SECONDARY/MIDDLE,GRADE 3,Kyrgyz,Osh city - urban,100.000000,4


In [ ]:
#rerunning now to show that because I set it to false, this didn't finalize. That's ok! I'm still working on this, and can continue cleaning this up for future PS's
grades.head(1)

,FS10,FS12,FS13,FS14,CB3,CB5B,FL3,FL7,FL9,FL20A,FL20B,FL23A,FL23B,FL25C,FL25D,FL25E,FL26,FL26A,FL26B,FL26C,FL26D,FL26E,FL27A,FL27B,FL27C,FL27D,FL27E,FL28,HH4,HH6,HH7,HH52,HL4,ED5A,ED5B,ethnicity,stratum,wrdprctcorrect,readingcomp
8,YES,KYRGYZ,KYRGYZ,KYRGYZ,7.0,GRADE 2,YES,KYRGYZ,RUSSIAN,71.0,2.0,CORRECT(9),CORRECT( 12 ),CORRECT (10),CORRECT (19),INCORRECT,CORRECT (3),CONTINUE,NaN,CORRECT (20),CONTINUE,NaN,CORRECT (8),CORRECT (16),CORRECT (30),CORRECT (8),CORRECT (14),COMPLETED,30.0,URBAN,Issyk-kul,2.0,FEMALE,HIGHER,GRADE 5,Other ethnicity,Issyk-kul - urban,97.183099,4


##RECAP - Data Set 1
Ok! I'm not fully finished cleaning this dataset, but I think for my first Problem Set I managed to get a lot handled.

1.   I've identified a majority of the variables I need to keep, and created the
2.   reading and reading comprehension percentage score
Have filtered the dataset to grade 2
3.   Have begun renaming the columns, still a bit more to do and need to finish reviewing

##Next Steps for this dataset:
1. Finalize Math inputs - there are several math subtasks that I need to consolidate, just like I did for reading. then I can drop them form the dataset.
2. Once that's done I can focus on final cleaning, and exporting this for a future merge in PS2 or PS3.

# Data Set 2 - UN Global Data Lab Population Information

In [ ]:
#now to read in a second dataset - let's grab some data from UNGLOBAL Data lab by region, it's quite clean and ready to go!
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8/export?format=csv' -O GDL-pop-data.csv

pop = pd.read_csv('/content/GDL-pop-data.csv')
pop.head(1)

--2025-02-04 23:21:13--  https://docs.google.com/spreadsheets/d/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8/export?format=csv
Resolving docs.google.com (docs.google.com)... 74.125.126.102, 74.125.126.138, 74.125.126.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.126.102|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-10-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/v28dlmgc016krc1ego15p1lda8/1738711270000/113781219181981321798/*/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8?format=csv [following]
--2025-02-04 23:21:13--  https://doc-10-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/v28dlmgc016krc1ego15p1lda8/1738711270000/113781219181981321798/*/1V9cMDdPdcf19ahKqUoT2uxJxsXas-qGYLSBD3QG_rL8?format=csv
Resolving doc-10-7s-sheets.googleusercontent.com (doc-10-7s-sheets.googleusercontent.com)... 142.251.183.132, 2607:f8b0:4001:c64::84
Connecting to doc-10-7s-sheets.googleusercont

,Country,Continent,ISOCode,Level,GDLCODE,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kyrgyz Republic,Asia/Pacific,KGZ,National,KGZt,Total,NaN,NaN,4.52,4.58,4.64,4.7,4.75,4.81,4.87,4.93,4.99,5.04,5.1,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.2,6.32,6.44,6.57,6.69,NaN,NaN


In [ ]:
pop.drop(columns=['Country', 'Continent', 'ISOCode', 'Level', 'GDLCODE'], inplace=True)
pop.head(10)

,Region,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Total,NaN,NaN,4.52,4.58,4.64,4.70,4.75,4.81,4.87,4.93,4.99,5.04,5.10,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.20,6.32,6.44,6.57,6.69,NaN,NaN
1,Batken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.42,0.42,0.42,0.42,0.42,0.42,0.43,0.43,0.43,0.43,0.45,0.48,0.47,0.47,0.46,0.46,0.46,0.45,0.45,NaN,NaN
2,Bishkek,NaN,NaN,0.42,0.46,0.50,0.54,0.58,0.62,0.66,0.70,0.74,0.78,0.82,0.86,0.90,0.88,0.87,0.85,0.84,0.83,0.81,0.78,0.75,0.86,0.97,1.08,1.19,1.31,1.42,1.53,NaN,NaN
3,Chuy,NaN,NaN,0.81,0.81,0.81,0.81,0.81,0.81,0.81,0.81,0.80,0.80,0.80,0.80,0.80,0.83,0.86,0.89,0.91,0.94,0.97,1.01,1.04,1.03,1.01,0.99,0.98,0.96,0.95,0.93,NaN,NaN
4,Issyk-Kul,NaN,NaN,0.46,0.45,0.45,0.45,0.44,0.44,0.43,0.43,0.42,0.42,0.42,0.41,0.41,0.42,0.44,0.45,0.47,0.48,0.50,0.47,0.44,0.45,0.47,0.48,0.50,0.51,0.53,0.54,NaN,NaN
5,Jalal-Abad,NaN,NaN,0.82,0.83,0.84,0.86,0.87,0.88,0.89,0.91,0.92,0.93,0.94,0.96,0.97,0.97,0.98,0.98,0.98,0.98,0.99,1.07,1.15,1.13,1.10,1.07,1.05,1.02,0.99,0.96,NaN,NaN
6,Naryn,NaN,NaN,0.27,0.27,0.27,0.27,0.26,0.26,0.26,0.26,0.25,0.25,0.25,0.25,0.24,0.25,0.25,0.25,0.25,0.26,0.26,0.27,0.28,0.28,0.29,0.29,0.30,0.30,0.31,0.31,NaN,NaN
7,Osh,NaN,NaN,1.68,1.64,1.61,1.58,1.54,1.51,1.47,1.44,1.41,1.37,1.34,1.30,1.27,1.29,1.30,1.32,1.34,1.35,1.37,1.39,1.41,1.45,1.49,1.53,1.57,1.62,1.66,1.70,NaN,NaN
8,Talas,NaN,NaN,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.21,0.21,0.21,0.21,0.21,0.21,0.22,0.23,0.25,0.26,0.27,0.28,0.29,0.30,0.29,0.29,0.28,0.28,0.27,0.27,0.26,NaN,NaN


I want to drop the NAN values, and calculate a Compound Average Growth Rate for population for each region. To do that I'll need to use data that's available for each yera, so, 2003 -2021.

In [ ]:
# Drop the NAN values as below
pop.dropna(axis=1, how='any', inplace=True)

# Function to calculate CAGR for each region (row)
def calculate_cagr(row):
    # Skip the first column (Region)
    years = len(row) - 1  # Exclude the Region column from the count
    start_value = row[1]  # Start value (first year data)
    end_value = row[years]  # End value (last year data)

    # If start_value or end_value is 0, return NaN
    if start_value == 0 or end_value == 0:
        return np.nan

    # Calculate CAGR
    try:
        cagr = ((end_value / start_value) ** (1 / years)) - 1
    except ZeroDivisionError:
        return np.nan  # Handle potential divide-by-zero error

    return round(cagr * 100, 2)  # Convert to percentage and round to two decimal places

# Apply the CAGR function to each row of the population DataFrame, excluding the Region column
pop['CAGR'] = pop.iloc[:, 1:].apply(calculate_cagr, axis=1)

# Display the updated DataFrame with the CAGR values
pop.head(10)

<ipython-input-68-7d4b1e2e6ad6>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start_value = row[1]  # Start value (first year data)
<ipython-input-68-7d4b1e2e6ad6>:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  end_value = row[years]  # End value (last year data)


,Region,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,CAGR
0,Total,5.04,5.10,5.16,5.22,5.28,5.35,5.41,5.48,5.54,5.61,5.72,5.84,5.96,6.08,6.20,6.32,6.44,6.57,6.69,1.52
1,Batken,0.42,0.42,0.42,0.42,0.42,0.42,0.43,0.43,0.43,0.43,0.45,0.48,0.47,0.47,0.46,0.46,0.46,0.45,0.45,0.38
2,Bishkek,0.78,0.82,0.86,0.90,0.88,0.87,0.85,0.84,0.83,0.81,0.78,0.75,0.86,0.97,1.08,1.19,1.31,1.42,1.53,3.53
3,Chuy,0.80,0.80,0.80,0.80,0.83,0.86,0.89,0.91,0.94,0.97,1.01,1.04,1.03,1.01,0.99,0.98,0.96,0.95,0.93,0.84
4,Issyk-Kul,0.42,0.42,0.41,0.41,0.42,0.44,0.45,0.47,0.48,0.50,0.47,0.44,0.45,0.47,0.48,0.50,0.51,0.53,0.54,1.41
5,Jalal-Abad,0.93,0.94,0.96,0.97,0.97,0.98,0.98,0.98,0.98,0.99,1.07,1.15,1.13,1.10,1.07,1.05,1.02,0.99,0.96,0.12
6,Naryn,0.25,0.25,0.25,0.24,0.25,0.25,0.25,0.25,0.26,0.26,0.27,0.28,0.28,0.29,0.29,0.30,0.30,0.31,0.31,1.20
7,Osh,1.37,1.34,1.30,1.27,1.29,1.30,1.32,1.34,1.35,1.37,1.39,1.41,1.45,1.49,1.53,1.57,1.62,1.66,1.70,1.33
8,Talas,0.21,0.21,0.21,0.21,0.22,0.23,0.25,0.26,0.27,0.28,0.29,0.30,0.29,0.29,0.28,0.28,0.27,0.27,0.26,1.19


In [ ]:
#need to convert the dataframe csv if I want to download this
pop.to_csv('pop.csv', index=False)

#let's download this to csv for data visualization PS0 or PS1!
files.download('pop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Recap - Dataset 2
We see that Bishkek had a CAGR throught he period 2003-2021 of 3.53%. This is more than double the total and any other region.
that is to say, while the percentages look small, Bishkek has far exceeded the actual percentage growth. the nominal numbers above are in millions.
So Bishkek doubled in size during this time period. most other places barely saw increases. Talk about everyone being packed into one small space!

# Dataset 3 - Kyrgyz Bureau of Statistics - Per Capita Income

In [ ]:
#Ok. Now need to convert a raw JSON file with per capita income monthly by region. I don't understand why downloading it led to a weird text file... Chatgpt lead the way!

# let's load the third data set - this was downloaded from Kyrgyzstan bureau of stats
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i' -O percapita.json
#Read in the raw JSON data directly from file
income = pd.read_json('/content/percapita.json')

# Check the structure of the loaded data to see how it is organized
print(income.head())

# Initialize a list to hold the rows of the DataFrame
rows = []

# where 'data' is the key containing the region data
for region in income['data']:
    region_name = region['title_en']  # Get the region name
    values = region['values']  # Get the list of values for this region

    # Create a dictionary with region as the first entry and year-value pairs following
    region_data = {'Region': region_name}
    for year_data in values:
        year = year_data['key']
        value = year_data['value']
        region_data[year] = value

    # Append the row to the list
    rows.append(region_data)

# Create a DataFrame from the rows
income_df = pd.DataFrame(rows)
# Reorder the columns so that years are in ascending order (from 2019 to 2023, etc.)
year_columns = sorted([col for col in income_df.columns if isinstance(col, int)])  # Get only the year columns
ordered_columns = ['Region'] + year_columns  # Place 'Region' as the first column, followed by years

# Reorder the DataFrame columns
income_df = income_df[ordered_columns]

# Display the DataFrame
income_df

--2025-02-08 18:06:41--  https://drive.google.com/uc?export=download&id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i
Resolving drive.google.com (drive.google.com)... 142.250.145.102, 142.250.145.113, 142.250.145.101, ...
Connecting to drive.google.com (drive.google.com)|142.250.145.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i&export=download [following]
--2025-02-08 18:06:41--  https://drive.usercontent.google.com/download?id=1kUbVCPCkv5G27fCeEYf_IxOlRqvL6y6i&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.218.132, 2a00:1450:4013:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.218.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3378 (3.3K) [application/octet-stream]
Saving to: ‘percapita.json’

percapita.json      100%[===================>]   3.30K  --.-K

,Region,2019,2020,2021,2022,2023
0,Kyrgyz Republic,5684.7,5625.4,6647.8,7948.4,9716.0
1,Batken oblast,5346.5,5386.9,6205.4,7531.8,9877.4
2,Jalal-Abat oblast,5293.2,5415.0,6672.2,8253.9,10327.6
3,Yssyk-Kul oblast,5719.5,6433.8,7173.2,9114.5,10105.3
4,Naryn oblast,4663.7,4588.5,5808.8,7096.2,8290.5
5,Osh oblast (until 2013 y. including Osh city),5914.4,5700.5,6922.4,7828.4,9512.8
6,Talas oblast,4834.5,5262.9,5732.5,6986.7,8406.2
7,Chui oblast,6116.3,5774.7,6755.1,7975.8,9298.0
8,Bishkek city,6340.8,6012.9,6946.2,8484.1,10767.1
9,Osh city,4892.3,4770.0,5523.1,5919.1,7581.1


In [ ]:
# Define the exchange rate
exchange_rate = 87.45

# Convert the whole DataFrame from KGS to USD
income_df_usd = income_df.copy()

# Apply the conversion for each year (for all regions)
for year in income_df.columns[1:]:  # Skip the 'Region' column
    income_df_usd[year] = income_df[year] / exchange_rate

# Display the converted DataFrame
income_df_usd.round(2).head(10)

,Region,2019,2020,2021,2022,2023
0,Kyrgyz Republic,65.01,64.33,76.02,90.89,111.10
1,Batken oblast,61.14,61.60,70.96,86.13,112.95
2,Jalal-Abat oblast,60.53,61.92,76.30,94.38,118.10
3,Yssyk-Kul oblast,65.40,73.57,82.03,104.23,115.56
4,Naryn oblast,53.33,52.47,66.42,81.15,94.80
5,Osh oblast (until 2013 y. including Osh city),67.63,65.19,79.16,89.52,108.78
6,Talas oblast,55.28,60.18,65.55,79.89,96.13
7,Chui oblast,69.94,66.03,77.25,91.20,106.32
8,Bishkek city,72.51,68.76,79.43,97.02,123.12
9,Osh city,55.94,54.55,63.16,67.69,86.69


In [ ]:
#It looks like the per capita income essentially doubled across the board from 2019 to 2023. Is that because of some larger changes in the country, or am I dealing with poor data? It is especially interesting to note how poor Osh City is on a per capita basis.
#convert dataframe to csv
income_df_usd.to_csv('income_usd.csv', index=False)

# Download the file, so I can use this for visual!
files.download('income_usd.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Recap -Dataset 3
This shows per capita income (in USD), which may be an imperfect measure, essentially doubling over a five year period. However, interestingly the wages shown are about a third of what I find online regarding average monthly salaries. This probably means I can read into this more - is there super high unemployment? Is there a super high percentage of youth and elderly bringing the per capita income down? Lots to explore there!

# Dataset 4 - Early Grade Reading Assessment Data

In [ ]:
# let's load the fourth data set - this is the EGRA file
!wget --no-check-certificate 'https://docs.google.com/spreadsheets/d/1Rq8nK25uEKsjwwcu0Sr1IF1oBD6N2BwbmFW5VeiGAfU/export?format=csv' -O Datareading.csv


--2025-02-08 18:44:48--  https://docs.google.com/spreadsheets/d/1Rq8nK25uEKsjwwcu0Sr1IF1oBD6N2BwbmFW5VeiGAfU/export?format=csv
Resolving docs.google.com (docs.google.com)... 108.177.96.113, 108.177.96.100, 108.177.96.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.96.113|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0c-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/1eu6jniqm0uvjqc4epgr0jv1ns/1739040285000/113781219181981321798/*/1Rq8nK25uEKsjwwcu0Sr1IF1oBD6N2BwbmFW5VeiGAfU?format=csv [following]
--2025-02-08 18:44:48--  https://doc-0c-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/1eu6jniqm0uvjqc4epgr0jv1ns/1739040285000/113781219181981321798/*/1Rq8nK25uEKsjwwcu0Sr1IF1oBD6N2BwbmFW5VeiGAfU?format=csv
Resolving doc-0c-7s-sheets.googleusercontent.com (doc-0c-7s-sheets.googleusercontent.com)... 74.125.128.132, 2a00:1450:4013:c02::84
Connecting to doc-0c-7s-sheets.googleuserconte

In [ ]:
egra = pd.read_csv('Datareading.csv')#let's take a look at this
egra.describe()
#oh man.


<ipython-input-83-5d8047de8398>:1: DtypeWarning: Columns (15,18,19,20,21,22,23,24,25,27,28,30,31,32,33,34,35,36,37,38,39,44,45,46,50,52,58,59,60,67,69,70,71,80,90,95,96,100,101,102,104,108,109,110,115,121,128,132,133,134,136,137,138,139,140,141,146,147,148,152,155,156,157,158,159,167,168,176,178,179,185,192,200,201,212,219,220,229,230,241,242,243,245,253,254,263,267,268,269,271,276,277,278,280,281,290,291,292,293,294,300,301,305,306,307,311,312,313,314,315,323,324,326,327,328,333,337,338,391,392,393,396,397,452,453,454,455,456,457,458,459,460,461,462,463,464,465,468,469,475,476,478,479,485,486,488,489,500,501,503,504,510,511,513,514,520,521,523,524,533,535,549,551,555,559,563,565,567,569,571,572,573,574,575,576,582,583,584,585,586,587,588,592,593,604,605,608,610,624,626,629,630,631,632,633,634,637,638,639,640,642,643,644,645,647,648,649,650,652,653,654,655,660,661,662,663,664,665,667,670,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696

,region,district,school_code,language,language_name,urbanrural,loi,school_name,treatment,loi_name,loi_kyrgyz,loi_russian,loi_uzbek,loi_tajik,stage1,fpc1,in_G2_master,in_G4_master,n_sampled,n_g4_sampled,pop_strata1,n_g2f_sampled,n_g2m_sampled,Dup_id,g_tot,n_classes,strata,n_g4f_sampled,n_g4m_sampled,grade,n_g2_sampled,pop_g2_strata1,pop_g4_strata1,g2g4_tot,pop_g2g4_strata1,g2_tot,g4_tot,g2_classes,g4_classes,in_ht,ht_start_time,ht_end_time,ht_time_start,ht_enumerator,ht_gps_latitude,ht_gps_longitude,ht_gps_accuracy,userprofileitem1first_name,userprofileitem1last_name,sh_consent,sh_female,sh_position,sh_position_other,sh_1,sh_2_1,sh_2_2,sh_2_3,sh_2_4,sh_2_777,sh_2_888,sh_2_other,sh_3_1,sh_3_2,sh_3_3,sh_3_4,sh_3_5,sh_3_6,sh_3_8,sh_3_7,sh_3_777,sh_3_888,sh_3_other,sh_4_1,sh_4_2,sh_4_3,sh_4_4,sh_4_5,sh_4_6,sh_4_7,sh_4_777,sh_4_888,sh_4_other,sh_5,sh_6,sh_7,sh_8_1,sh_8_2,sh_8_3,sh_8_4,sh_8_5,sh_8_888,sh_9,sh_10,date_end,ht_time_end,sh_8_6,ht_year,ht_date,ht_date_char,ht_dayofweek,ht_month,ht_day,sh_minutes_assess,_merge_Master_HT,in_l,l_start_time,l_end_time,l_enumerator,l_gps_latitude,l_gps_longitude,l_gps_accuracy,l_consent,l_female,l1,l2,l3,l4,l5_1,l5_2,l5_3,l5_4,l5_888,l6,l7,l8,l9,l_time_end,l_time_start,l_year,l_date,l_date_char,l_dayofweek,l_month,l_day,l_minutes_assess,_merge_Master_Lib,n_schools_sampled,wt1,sum_wt1,scale_wt1,wt_stage1,in_t,t_start_time,t_end_time,t_time_start,t_enumerator,t_gps_latitude,t_gps_longitude,t_gps_accuracy,t_consent,t_female,tq_language,tq_language_other,tq_1,tq_2,tq_3,tq_4_1,tq_4_2,tq_4_3,tq_4_888,tq_5,tq_6_1,tq_6_2,tq_6_3,tq_6_4,tq_6_5,tq_6_6,tq_6_000,tq_6_888,tq_7,tq_8,tq_9,tq_10_1,tq_10_2,tq_10_3,tq_10_4,tq_10_5,tq_10_6,tq_10_000,tq_10_888,tq_11_1,tq_11_2,tq_11_3,tq_11_4,tq_11_777,tq_11_888,tq_11_other,tq_12_1,tq_12_2,tq_12_3,tq_12_4,tq_12_5,tq_12_888,tq_13_1,tq_13_2,tq_13_3,tq_13_4,tq_13_5,tq_13_6,tq_13_7,tq_13_000,tq_13_888,tq_14,tq_15,tq_16,tq_17_1,tq_17_2,tq_17_3,tq_17_4,tq_17_5,tq_17_6,tq_17_777,tq_17_888,tq_17_other,tq_18_1,tq_18_2,tq_18_3,tq_18_4,tq_18_777,tq_18_000,tq_18_888,tq_18_other,tq_19,tq_20_1,tq_20_2,tq_20_3,tq_20_4,tq_20_5,tq_20_6,tq_20_777,tq_20_888,tq_21,tq_21_other,tq_22,tq_23_1,tq_23_2,tq_23_3,tq_23_4,tq_23_5,tq_23_6,tq_23_7,tq_23_777,tq_23_888,tq_23_other,tq_24,tq_24_other,tq_25_1,tq_25_2,tq_25_3,tq_25_4,tq_25_5,tq_25_6,tq_25_7,tq_25_777,tq_25_888,tq_26,tq_27,tq_28,tq_29,tq_30,tq_31,tq_32,t_time_end,t_year,t_date,t_date_char,t_dayofweek,t_month,t_day,t_minutes_assess,_merge_Master_Teach,in_ci,ci_start_time,ci_end_time,ci_time_start,ci_enumerator,ci_gps_latitude,ci_gps_longitude,ci_gps_accuracy,ci_language,males_enrolled,females_enrolled,ci_1_1,ci_1_2,ci_1_3,ci_1_4,ci_1_5,ci_1_6,ci_1_7,ci_1_8,ci_1_000,ci_2,ci_3,ci_4,ci_5,ci_6,ci_7,ci_8,ci_9,ci_time_end,userprofileid,ci_year,ci_date,ci_date_char,ci_dayofweek,ci_month,ci_day,ci_minutes_assess,_merge_Master_ClassInv,strata2,stage2,fpc2,wt2,wt12,wt_stage2,in_g2,sil_read_comp_score_pcnt80,sil_read_comp_attempted_pcnt80,sil_read_comp1,sil_read_comp2,sil_read_comp3,sil_read_comp4,sil_read_comp5,sil_read_comp_score,sil_read_comp_score_pcnt,sil_read_comp_score_zero,sil_read_comp_attempted,sil_read_comp_attempted_pcnt,g2_year,g2_month,g2_date,id,female,age,g2_start_time,g2_end_time,g2_consent,cnonwpm,orf,read_comp_score_pcnt80,read_comp_attempted_pcnt80,invent_word1,invent_word2,invent_word3,invent_word4,invent_word5,invent_word6,invent_word7,invent_word8,invent_word9,invent_word10,invent_word11,invent_word12,invent_word13,invent_word14,invent_word15,invent_word16,invent_word17,invent_word18,invent_word19,invent_word20,invent_word21,invent_word22,invent_word23,invent_word24,invent_word25,invent_word26,invent_word27,invent_word28,invent_word29,invent_word30,invent_word31,invent_word32,invent_word33,invent_word34,invent_word35,invent_word36,invent_word37,invent_word38,invent_word39,invent_word40,invent_word41,invent_word42,invent_word43,invent_word44,invent_word45,invent_word46,invent_word47,invent_word48,invent_word49,invent_word50,i

In [ ]:
egra.head(2)
#One thing I really like about the way my colleague set this up, is there is a dsecription of each column in index 0. This makes it a LOT easier to figure out what to keep!

,region,district,school_code,language,language_name,urbanrural,loi,school_name,treatment,loi_name,loi_kyrgyz,loi_russian,loi_uzbek,loi_tajik,stage1,fpc1,in_G2_master,in_G4_master,n_sampled,n_g4_sampled,pop_strata1,n_g2f_sampled,n_g2m_sampled,Dup_id,g_tot,n_classes,strata,n_g4f_sampled,n_g4m_sampled,grade,n_g2_sampled,pop_g2_strata1,pop_g4_strata1,g2g4_tot,pop_g2g4_strata1,g2_tot,g4_tot,g2_classes,g4_classes,in_ht,ht_start_time,ht_end_time,ht_time_start,ht_enumerator,ht_gps_latitude,ht_gps_longitude,ht_gps_accuracy,userprofileitem1first_name,userprofileitem1last_name,sh_consent,sh_female,sh_position,sh_position_other,sh_1,sh_2_1,sh_2_2,sh_2_3,sh_2_4,sh_2_777,sh_2_888,sh_2_other,sh_3_1,sh_3_2,sh_3_3,sh_3_4,sh_3_5,sh_3_6,sh_3_8,sh_3_7,sh_3_777,sh_3_888,sh_3_other,sh_4_1,sh_4_2,sh_4_3,sh_4_4,sh_4_5,sh_4_6,sh_4_7,sh_4_777,sh_4_888,sh_4_other,sh_5,sh_6,sh_7,sh_8_1,sh_8_2,sh_8_3,sh_8_4,sh_8_5,sh_8_888,sh_9,sh_10,date_end,ht_time_end,sh_8_6,ht_year,ht_date,ht_date_char,ht_dayofweek,ht_month,ht_day,sh_minutes_assess,_merge_Master_HT,in_l,l_start_time,l_end_time,l_enumerator,l_gps_latitude,l_gps_longitude,l_gps_accuracy,l_consent,l_female,l1,l2,l3,l4,l5_1,l5_2,l5_3,l5_4,l5_888,l6,l7,l8,l9,l_time_end,l_time_start,l_year,l_date,l_date_char,l_dayofweek,l_month,l_day,l_minutes_assess,_merge_Master_Lib,n_schools_sampled,wt1,sum_wt1,scale_wt1,wt_stage1,in_t,t_start_time,t_end_time,t_time_start,t_enumerator,t_gps_latitude,t_gps_longitude,t_gps_accuracy,t_consent,t_female,tq_language,tq_language_other,tq_1,tq_2,tq_3,tq_4_1,tq_4_2,tq_4_3,tq_4_888,tq_5,tq_6_1,tq_6_2,tq_6_3,tq_6_4,tq_6_5,tq_6_6,tq_6_000,tq_6_888,tq_7,tq_8,tq_9,tq_10_1,tq_10_2,tq_10_3,tq_10_4,tq_10_5,tq_10_6,tq_10_000,tq_10_888,tq_11_1,tq_11_2,tq_11_3,tq_11_4,tq_11_777,tq_11_888,tq_11_other,tq_12_1,tq_12_2,tq_12_3,tq_12_4,tq_12_5,tq_12_888,tq_13_1,tq_13_2,tq_13_3,tq_13_4,tq_13_5,tq_13_6,tq_13_7,tq_13_000,tq_13_888,tq_14,tq_15,tq_16,tq_17_1,tq_17_2,tq_17_3,tq_17_4,tq_17_5,tq_17_6,tq_17_777,tq_17_888,tq_17_other,tq_18_1,tq_18_2,tq_18_3,tq_18_4,tq_18_777,tq_18_000,tq_18_888,tq_18_other,tq_19,tq_20_1,tq_20_2,tq_20_3,tq_20_4,tq_20_5,tq_20_6,tq_20_777,tq_20_888,tq_21,tq_21_other,tq_22,tq_23_1,tq_23_2,tq_23_3,tq_23_4,tq_23_5,tq_23_6,tq_23_7,tq_23_777,tq_23_888,tq_23_other,tq_24,tq_24_other,tq_25_1,tq_25_2,tq_25_3,tq_25_4,tq_25_5,tq_25_6,tq_25_7,tq_25_777,tq_25_888,tq_26,tq_27,tq_28,tq_29,tq_30,tq_31,tq_32,t_time_end,t_year,t_date,t_date_char,t_dayofweek,t_month,t_day,t_minutes_assess,_merge_Master_Teach,in_ci,ci_start_time,ci_end_time,ci_time_start,ci_enumerator,ci_gps_latitude,ci_gps_longitude,ci_gps_accuracy,ci_language,males_enrolled,females_enrolled,ci_1_1,ci_1_2,ci_1_3,ci_1_4,ci_1_5,ci_1_6,ci_1_7,ci_1_8,ci_1_000,ci_2,ci_3,ci_4,ci_5,ci_6,ci_7,ci_8,ci_9,ci_time_end,userprofileid,ci_year,ci_date,ci_date_char,ci_dayofweek,ci_month,ci_day,ci_minutes_assess,_merge_Master_ClassInv,strata2,stage2,fpc2,wt2,wt12,wt_stage2,in_g2,sil_read_comp_score_pcnt80,sil_read_comp_attempted_pcnt80,sil_read_comp1,sil_read_comp2,sil_read_comp3,sil_read_comp4,sil_read_comp5,sil_read_comp_score,sil_read_comp_score_pcnt,sil_read_comp_score_zero,sil_read_comp_attempted,sil_read_comp_attempted_pcnt,g2_year,g2_month,g2_date,id,female,age,g2_start_time,g2_end_time,g2_consent,cnonwpm,orf,read_comp_score_pcnt80,read_comp_attempted_pcnt80,invent_word1,invent_word2,invent_word3,invent_word4,invent_word5,invent_word6,invent_word7,invent_word8,invent_word9,invent_word10,invent_word11,invent_word12,invent_word13,invent_word14,invent_word15,invent_word16,invent_word17,invent_word18,invent_word19,invent_word20,invent_word21,invent_word22,invent_word23,invent_word24,invent_word25,invent_word26,invent_word27,invent_word28,invent_word29,invent_word30,invent_word31,invent_word32,invent_word33,invent_word34,invent_word35,invent_word36,invent_word37,invent_word38,invent_word39,invent_word40,invent_word41,invent_word42,invent_word43,invent_word44,invent_word45,invent_word46,invent_word47,invent_word48,invent_word49,invent_word50,i

In [ ]:
#let's check what we have. I know there's baseline and endline data here. We want just the baseline data, prior to program interventions
egra['g2_year'].value_counts()


,count
g2_year,
2024,2692
2021,1559
2021,1023
Grade 2 Read/Math Year,1


In [ ]:
#Strange. I have two identical values for 2021. what's up?
egra['g2_year'].apply(type)
#Looks like I have strings and integers...I want to convery this all to string


,g2_year
0,<class 'str'>
1,<class 'str'>
2,<class 'str'>
3,<class 'str'>
4,<class 'str'>
...,...
5270,<class 'int'>
5271,<class 'int'>
5272,<class 'int'>
5273,<class 'int'>


In [ ]:
#let's see if we can convert all our "2021"'s to a string so its the same value
egra['g2_year'] = egra['g2_year'].apply(lambda x: '2021' if str(x) == '2021' else x)


In [ ]:
#let's run it again, looks like it worked!
egra['g2_year'].value_counts()


,count
g2_year,
2024,2692
2021,2582
Grade 2 Read/Math Year,1


In [ ]:
#now I can finally filter to the data I want.
baseline = egra[egra['g2_year'] == '2021']

In [ ]:
baseline.count()
#bingo!

,0
region,2582
district,2582
school_code,2582
language,2582
language_name,2582
...,...
invent_worditem_at_time,0
oral_readitem_at_time,0
sil_readitem_at_time,0
userprofileitem1female,0


In [ ]:
baseline.groupby('region').size()
baseline.groupby('district').size()
#Breakdown by region, and by group within the baseline study of 2500 observations.
#will need to rename this, but gives some nice quick insights as to what i'll be looking at for my roll up by region!


,0
region,
Баткенская,301
Жалал-Абадская,580
Иссык-Кульская,230
Нарынская,96
Ошская,437
Таласская,140
Чуйская,341
г. Бишкек,318
г. Ош,139


,0
district,
Ак-Суйский,59
Аксыйский,25
Ала-Букинский,25
Алайский,10
Араванский,78
Ат-Башинский,18
Базар-Коргонский,98
Бакай-Атинский,30
Баткенский,30


##Recap - Dataset 4
I need to determine precisely what i need here. It should be region - which all need to be renamed into english script for future merges.

I also need to select the language used, if it was urban or rural. and language of instruction + Gender

Finally, I need to get the full scores for all EGRA subtasks used, Reading, Reading Comprehension, Invented Word Reading,missing number, add, sub, relational reasoning, spatial thinking. Drop all other columns



# Dataset 5 - INFORM Disaster Incidence by Region

In [ ]:
# let's load the fifth data set - this is the disaster incidence file
!wget --no-check-certificate "https://docs.google.com/spreadsheets/d/1CMnd_Q6dslsVYN3u1ueaugox47WUwEFQr2R_cqomBys/export?format=csv&gid=16033786" -O disaster.csv


--2025-02-08 18:56:18--  https://docs.google.com/spreadsheets/d/1CMnd_Q6dslsVYN3u1ueaugox47WUwEFQr2R_cqomBys/export?format=csv&gid=16033786
Resolving docs.google.com (docs.google.com)... 108.177.96.139, 108.177.96.138, 108.177.96.113, ...
Connecting to docs.google.com (docs.google.com)|108.177.96.139|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0g-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/fh9eludhvqve66ac8sj2hp090o/1739040975000/113781219181981321798/*/1CMnd_Q6dslsVYN3u1ueaugox47WUwEFQr2R_cqomBys?format=csv&gid=16033786 [following]
--2025-02-08 18:56:19--  https://doc-0g-7s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/fh9eludhvqve66ac8sj2hp090o/1739040975000/113781219181981321798/*/1CMnd_Q6dslsVYN3u1ueaugox47WUwEFQr2R_cqomBys?format=csv&gid=16033786
Resolving doc-0g-7s-sheets.googleusercontent.com (doc-0g-7s-sheets.googleusercontent.com)... 74.125.128.132, 2a00:1450:4013:c02::84
Connect

In [ ]:
disaster = pd.read_csv('disaster.csv')
disaster
disaster.describe()
#I need to do a lot with this dataset! Firstly, all of the actual column titles are on index 0 - this needs to be shifted up somehow.
#secondly, I need to drop everything that isn't kyrgyzstan, as that's our subject of interest.
#Thirdly, there is a description tab for each of the ratings and descriptions. Selecting the incidence of flood, landslide, drought, natural disaster, and infrastructure for each region sounds perfect!


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,COUNTRY,FIRST ADMINISTRATIVE LEVEL,ISO3+PCODE,Earthquake,Flood,Landslide,Drought,Natural,Projected conflict risk,Current conflict,Human,HAZARD & EXPOSURE,Development & Deprivation,Inequality,Aid Dependency,Socio-Economic Vulnerability,Uprooted people,Health Conditions,Children U5,Recent Shocks,Food Security,Vulnerable Groups,VULNERABILITY,Governance,Economy,Humanitarian,DRR,Institutional,Communication,Physical infrastructure,Access to health care,Infrastructure,LACK OF COPING CAPACITY,INFORM RISK,RISK CLASS,Rank,Reliability Index (*),Number of Missing Datasets,% of Missing Datasets,Recentness data (average years),Ratio of subnational vs national data
1,(a-z),(a-z),(a-z),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(V.Low-V.High),(1-83),(0-10),(0-54),(0-100%),(0-...),(0-...)
2,Armenia,Aragatsotn,AM02,6.5,4.2,0.0,3.8,4.0,6.9,7.6,7.6,6.1,1.5,6.4,3.5,3.2,5.1,4.0,4.8,0.0,2.7,4.2,3.7,5.1,5.0,6.1,4.0,5.1,4.6,2.0,7.9,4.8,5.0,4.8,Medium,32,6.0,1,2%,0.75,0.97
3,Armenia,Ararat,AM03,6.5,8.0,0.0,5.5,5.6,6.9,7.6,7.6,6.7,1.3,5.9,3.5,3.0,5.5,4.0,0.9,0.0,2.7,3.5,3.3,5.1,5.0,5.2,4.0,4.8,4.2,1.7,7.1,4.3,4.6,4.7,Medium,33,6.0,1,2%,0.75,0.97
4,Armenia,Armavir,AM04,6.9,6.0,0.0,7.5,5.7,6.9,7.6,7.6,6.8,1.2,8.0,3.5,3.5,5.1,4.0,1.6,0.0,2.7,3.5,3.5,5.1,5.0,5.0,4.0,4.8,4.0,0.7,7.8,4.2,4.5,4.7,Medium,33,6.0,1,2%,0.75,0.97
5,Armenia,Gegharkunik,AM05,8.9,3.2,0.5,3.6,5.0,6.9,7.6,7.6,6.5,1.4,7.7,3.5,3.5,4.2,4.0,0.0,0.0,2.7,2.9,3.2,5.1,5.0,5.1,4.0,4.8,4.2,2.9,6.8,4.6,4.7,4.6,Medium,35,6.0,1,2%,0.75,0.97
6,Armenia,Kotayk,AM07,8.9,3.8,3.7,4.1,5.7,6.9,7.6,7.6,6.8,1.5,5.0,3.5,2.9,8.5,4.0,0.0,0.0,2.7,4.7,3.9,5.1,5.0,5.4,4.0,4.9,3.7,1.7,6.0,3.8,4.4,4.9,Medium,28,6.0,1,2%,0.75,0.97
7,Armenia,Lori,AM06,8.8,5.2,1.2,2.5,5.3,6.9,7.6,7.6,6.6,1.5,6.8,3.5,3.3,4.1,4.0,0.4,0.0,2.7,2.9,3.1,5.1,5.0,3.8,4.0,4.5,3.9,2.1,6.1,4.0,4.3,4.4,Medium,39,6.0,1,2%,0.75,0.97
8,Armenia,Shirak,AM08,8.9,3.8,0.0,3.5,5.0,6.9,7.6,7.6,6.5,1.3,6.4,3.5,3.1,3.5,4.0,0.8,10.0,2.7,5.6,4.5,5.1,5.0,3.8,4.0,4.5,4.1,1.9,6.1,4.0,4.3,5.0,Medium,25,6.0,1,2%,0.75,0.97
9,Armenia,Syunik,AM09,8.6,6.1,9.2,0.5,7.1,6.9,7.6,7.6,7.4,1.3,6.3,3.5,3.1,9.6,4.0,1.5,0.0,2.7,5.7,4.5,5.1,5.0,3.9,4.0,4.5,4.2,2.4,8.1,4.9,4.7,5.4,High,12,6.0,1,2%,0.75,0.97


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
count,85,86,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,85,84,85
unique,10,86,85,40,50,33,35,49,8,12,16,47,40,46,12,32,23,35,29,15,10,41,41,10,40,45,8,38,41,38,39,34,30,39,6,39,18,11,11,10,14
top,Kazakhstan,FIRST ADMINISTRATIVE LEVEL,ISO3+PCODE,0.1,5.3,0.0,0.0,5.0,0.4,4.4,0.2,5.7,1.3,3.9,0.2,3.5,0.6,3.0,0.2,0.0,0.4,2.0,2.4,4.8,5.0,1.6,x,3.7,2.7,3.2,5.3,3.9,4.0,4.4,Medium,39,2.8,1,2%,0.55,1.40
freq,17,1,1,13,6,48,10,6,23,17,17,5,8,5,17,8,19,14,17,45,17,9,6,17,11,10,21,12,9,8,9,8,10,5,26,5,14,34,34,22,15


In [ ]:
#let's roll the index 0 up to the column names. Set columns to index 0
disaster.columns = disaster.iloc[0]
#now we need to drop index 0 data entirely
disaster = disaster.drop(0)
#rerun and let's see if it worked this time!
disaster

,COUNTRY,FIRST ADMINISTRATIVE LEVEL,ISO3+PCODE,Earthquake,Flood,Landslide,Drought,Natural,Projected conflict risk,Current conflict,Human,HAZARD & EXPOSURE,Development & Deprivation,Inequality,Aid Dependency,Socio-Economic Vulnerability,Uprooted people,Health Conditions,Children U5,Recent Shocks,Food Security,Vulnerable Groups,VULNERABILITY,Governance,Economy,Humanitarian,DRR,Institutional,Communication,Physical infrastructure,Access to health care,Infrastructure,LACK OF COPING CAPACITY,INFORM RISK,RISK CLASS,Rank,Reliability Index (*),Number of Missing Datasets,% of Missing Datasets,Recentness data (average years),Ratio of subnational vs national data
1,(a-z),(a-z),(a-z),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(0-10),(V.Low-V.High),(1-83),(0-10),(0-54),(0-100%),(0-...),(0-...)
2,Armenia,Aragatsotn,AM02,6.5,4.2,0.0,3.8,4.0,6.9,7.6,7.6,6.1,1.5,6.4,3.5,3.2,5.1,4.0,4.8,0.0,2.7,4.2,3.7,5.1,5.0,6.1,4.0,5.1,4.6,2.0,7.9,4.8,5.0,4.8,Medium,32,6.0,1,2%,0.75,0.97
3,Armenia,Ararat,AM03,6.5,8.0,0.0,5.5,5.6,6.9,7.6,7.6,6.7,1.3,5.9,3.5,3.0,5.5,4.0,0.9,0.0,2.7,3.5,3.3,5.1,5.0,5.2,4.0,4.8,4.2,1.7,7.1,4.3,4.6,4.7,Medium,33,6.0,1,2%,0.75,0.97
4,Armenia,Armavir,AM04,6.9,6.0,0.0,7.5,5.7,6.9,7.6,7.6,6.8,1.2,8.0,3.5,3.5,5.1,4.0,1.6,0.0,2.7,3.5,3.5,5.1,5.0,5.0,4.0,4.8,4.0,0.7,7.8,4.2,4.5,4.7,Medium,33,6.0,1,2%,0.75,0.97
5,Armenia,Gegharkunik,AM05,8.9,3.2,0.5,3.6,5.0,6.9,7.6,7.6,6.5,1.4,7.7,3.5,3.5,4.2,4.0,0.0,0.0,2.7,2.9,3.2,5.1,5.0,5.1,4.0,4.8,4.2,2.9,6.8,4.6,4.7,4.6,Medium,35,6.0,1,2%,0.75,0.97
6,Armenia,Kotayk,AM07,8.9,3.8,3.7,4.1,5.7,6.9,7.6,7.6,6.8,1.5,5.0,3.5,2.9,8.5,4.0,0.0,0.0,2.7,4.7,3.9,5.1,5.0,5.4,4.0,4.9,3.7,1.7,6.0,3.8,4.4,4.9,Medium,28,6.0,1,2%,0.75,0.97
7,Armenia,Lori,AM06,8.8,5.2,1.2,2.5,5.3,6.9,7.6,7.6,6.6,1.5,6.8,3.5,3.3,4.1,4.0,0.4,0.0,2.7,2.9,3.1,5.1,5.0,3.8,4.0,4.5,3.9,2.1,6.1,4.0,4.3,4.4,Medium,39,6.0,1,2%,0.75,0.97
8,Armenia,Shirak,AM08,8.9,3.8,0.0,3.5,5.0,6.9,7.6,7.6,6.5,1.3,6.4,3.5,3.1,3.5,4.0,0.8,10.0,2.7,5.6,4.5,5.1,5.0,3.8,4.0,4.5,4.1,1.9,6.1,4.0,4.3,5.0,Medium,25,6.0,1,2%,0.75,0.97
9,Armenia,Syunik,AM09,8.6,6.1,9.2,0.5,7.1,6.9,7.6,7.6,7.4,1.3,6.3,3.5,3.1,9.6,4.0,1.5,0.0,2.7,5.7,4.5,5.1,5.0,3.9,4.0,4.5,4.2,2.4,8.1,4.9,4.7,5.4,High,12,6.0,1,2%,0.75,0.97
10,Armenia,Tavush,AM11,6.9,4.6,6.8,3.9,5.7,6.9,7.6,7.6,6.8,1.3,6.0,3.5,3.0,4.0,4.0,0.3,0.0,2.7,2.9,3.0,5.1,5.0,1.6,4.0,3.9,4.3,2.3,6.8,4.5,4.2,4.4,Medium,39,6.0,1,2%,0.75,0.97


In [ ]:
#the scale is nice, but not necessary in index 1. let's drop it
disaster = disaster.drop(1)


In [ ]:
disaster.rename(columns={'FIRST ADMINISTRATIVE LEVEL' : 'region', 'COUNTRY':'country'}, inplace=True)

In [ ]:

#ok, now let's filter to this the country I'm interested in
disasterkgz = disaster[disaster["country"] == "Kyrgyzstan"]
disasterkgz
#filtered to kgz. Still have a lot of work to do to understand the ratings, rename each region, and drop a bunch of the things I don't need. but let's save that for PS2

,country,region,ISO3+PCODE,Earthquake,Flood,Landslide,Drought,Natural,Projected conflict risk,Current conflict,Human,HAZARD & EXPOSURE,Development & Deprivation,Inequality,Aid Dependency,Socio-Economic Vulnerability,Uprooted people,Health Conditions,Children U5,Recent Shocks,Food Security,Vulnerable Groups,VULNERABILITY,Governance,Economy,Humanitarian,DRR,Institutional,Communication,Physical infrastructure,Access to health care,Infrastructure,LACK OF COPING CAPACITY,INFORM RISK,RISK CLASS,Rank,Reliability Index (*),Number of Missing Datasets,% of Missing Datasets,Recentness data (average years),Ratio of subnational vs national data
34,Kyrgyzstan,Batken,KG41705,5.4,5.7,7.5,7.2,6.5,6.4,4.4,5.5,6.0,3.3,5.3,6.7,4.7,0.5,2.7,3.6,x,3.9,2.8,3.8,6.4,10.0,6.0,6.4,7.2,2.7,5.7,7.7,5.4,6.4,5.3,High,14,3.7,1,2%,0.72,1.31
35,Kyrgyzstan,Bishkek (city),KG41711,6.5,6.4,0.0,0.0,3.9,6.4,7.0,7.0,5.7,2.1,4.3,6.7,3.8,0.5,8.4,1.7,x,3.9,4.5,4.2,6.4,9.1,3.4,6.4,6.3,2.7,0.0,7.2,3.3,5.0,4.9,Medium,28,3.7,1,2%,0.72,1.31
36,Kyrgyzstan,Chui,KG41708,6.8,5.0,8.6,2.0,6.2,6.4,4.4,5.5,5.9,2.5,5.2,6.7,4.2,0.5,7.0,3.1,x,3.9,4.0,4.1,6.4,9.7,6.6,6.4,7.3,2.7,2.6,8.1,4.5,6.1,5.3,High,14,3.7,1,2%,0.72,1.31
37,Kyrgyzstan,Issyk-Kul,KG41702,8.2,4.7,9.1,2.5,6.9,6.4,7.0,7.0,7.0,3.0,4.8,6.7,4.4,0.5,3.4,2.8,x,3.9,2.7,3.6,6.4,9.4,4.3,6.4,6.6,2.7,3.2,7.9,4.6,5.7,5.2,Medium,18,3.7,1,2%,0.72,1.31
38,Kyrgyzstan,Jalal-Abad,KG41703,9.4,7.3,9.4,9.3,9.0,6.4,7.0,7.0,8.2,3.3,5.4,6.7,4.7,0.5,3.0,1.9,x,3.9,2.4,3.6,6.4,9.9,5.2,6.4,7.0,2.7,4.3,8.4,5.1,6.1,5.6,High,7,3.7,1,2%,0.72,1.31
39,Kyrgyzstan,Naryn,KG41704,5.9,5.2,9.1,2.0,6.2,6.4,4.4,5.5,5.9,2.9,4.5,6.7,4.3,0.5,3.3,5.3,x,3.9,3.4,3.9,6.4,10.0,4.2,6.4,6.8,2.7,3.3,7.2,4.4,5.7,5.1,Medium,21,3.7,1,2%,0.72,1.31
40,Kyrgyzstan,Osh,KG41706,9.5,6.5,9.5,7.7,8.6,6.4,7.0,7.0,7.9,3.1,5.1,6.7,4.5,0.5,3.2,2.7,x,3.9,2.7,3.7,6.4,10.0,5.5,6.4,7.1,2.7,4.0,7.7,4.8,6.1,5.6,High,7,3.7,1,2%,0.72,1.31
41,Kyrgyzstan,Osh (city),KG41721,8.9,7.0,0.0,9.2,7.4,6.4,7.0,7.0,7.2,6.6,5.4,6.7,6.3,0.5,5.5,3.1,x,3.9,3.5,5.1,6.4,9.6,8.7,6.4,7.8,2.7,0.1,6.3,3.0,5.9,6.0,High,5,4.3,3,6%,0.72,1.31
42,Kyrgyzstan,Talas,KG41707,5.9,5.6,2.5,6.4,5.3,6.4,7.0,7.0,6.2,3.5,4.5,6.7,4.6,0.5,3.0,0.3,x,3.9,2.1,3.5,6.4,9.9,3.4,6.4,6.5,2.7,2.9,7.2,4.3,5.5,4.9,Medium,28,3.7,1,2%,0.72,1.31


In [ ]:
#let's get basic stats on the regions in kyrgyzstan, and their likelyhood on a scale of 1-10 to deal with earth quakes
disasterkgz['Earthquake'].describe()


,Earthquake
count,9
unique,8
top,5.9
freq,2


In [ ]:
#let's look now just at the region and the incidence rating.
disasterkgz[['region', 'Earthquake']].sort_values(by='Earthquake', ascending=True)
#looks like Jalal abad and Osh are at super high risk of earthquakes. Might be worth reading more on those areas and their population! Good thing I have some of that data :-)


,region,Earthquake
34,Batken,5.4
39,Naryn,5.9
42,Talas,5.9
35,Bishkek (city),6.5
36,Chui,6.8
37,Issyk-Kul,8.2
41,Osh (city),8.9
38,Jalal-Abad,9.4
40,Osh,9.5


##Recap Dataset 5
Ok! We have this filtered to Kyrgyzstan, and proper column headers. What I really need to do now is cut this down for a future problem set so that I have just the disaster incidence data for the five natural disasters. This will be an easy clean and merge onto the larger dataset for education, demographic, and income data.

#bonus section. working on pdf to dataframe imports with tabula.
I gathered from google I can use tabula to parse pdf's and import tables into python. but when reading in a page it's an absolute mess. My initial thought is it'd be a lot easier to take the values and convert this into one, individual dataframe for all the tables at once with manual entries.

this being data management, I guessed that wouldn't t fly and I needed to figure this out! Coming back to this text comment now that i've spent the last hour or so googling and checking how to get tabula to organize this into lists.

 I'll have to do this operation 9 time (for each individual region), then merge it all together on a new column I would call "country", so that way all the individual tables, data, and information are provided. Good news is - below I have proof of concept and cracked the case on pdf to pandas dataframe conversion. Now the trick is repeating this process for each of the "list" tables, cleaning, converting, then merging on the added "country" column.

 Exciting, though this is going to be a lot of work.



In [26]:
# Check if the file exists
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1BQ7RpjsBdBTHx-qqBUDu6WaukjNza2aq" -O nst2021.pdf


--2025-02-10 20:29:04--  https://drive.google.com/uc?export=download&id=1BQ7RpjsBdBTHx-qqBUDu6WaukjNza2aq
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.113, 74.125.142.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1BQ7RpjsBdBTHx-qqBUDu6WaukjNza2aq&export=download [following]
--2025-02-10 20:29:04--  https://drive.usercontent.google.com/download?id=1BQ7RpjsBdBTHx-qqBUDu6WaukjNza2aq&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.188.132, 2607:f8b0:400e:c1b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.188.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11199477 (11M) [application/octet-stream]
Saving to: ‘nst2021.pdf’

nst2021.pdf         100%[===================>]  10.68M  --.-KB/s 

In [27]:
Nat = read_pdf('nst2021.pdf', pages='25-29',multiple_tables=True, stream=True)
Nat
#well. the stream function helps but this is still horrifying. I need to figure out how to essentially take the first two tables from each, combine them, then add country + region. Then repeat that process 8 or 9 times, then merge, all while double checking the pdf to ensure this matches the reported data


[     Unnamed: 0    Unnamed: 1    Unnamed: 2 стандартноe      Unnamed: 3  \
 0          язык    количество  средн. знач.  отклонение     коэффициент   
 1  тестирования    участников      тестовых    тестовых      надежности   
 2           NaN  тестирования        баллов         NaN  Альфа Кронбаха   
 3           NaN           NaN           NaN      баллов             NaN   
 4    Кыргызский         27500         114.0        29.2            0.91   
 5       Русский         17520         130.7        33.4            0.93   
 6      В целом:         45020         120.5        32.0            0.93   
 
      Unnamed: 4    Unnamed: 5    Unnamed: 6  
 0          язык    количество  средн. знач.  
 1  тестирования   зачисленных      тестовых  
 2           NaN  абитуриентов        баллов  
 3           NaN           NaN           NaN  
 4    Кыргызский          2877         145.1  
 5       Русский          1248         156.7  
 6      В целом:          4125         148.6  ,
      Unnamed

In [28]:
print(type(Nat[0]))  # Check the type of the first element
print(Nat[0])        # Print the first element


<class 'pandas.core.frame.DataFrame'>
     Unnamed: 0    Unnamed: 1    Unnamed: 2 стандартноe      Unnamed: 3  \
0          язык    количество  средн. знач.  отклонение     коэффициент   
1  тестирования    участников      тестовых    тестовых      надежности   
2           NaN  тестирования        баллов         NaN  Альфа Кронбаха   
3           NaN           NaN           NaN      баллов             NaN   
4    Кыргызский         27500         114.0        29.2            0.91   
5       Русский         17520         130.7        33.4            0.93   
6      В целом:         45020         120.5        32.0            0.93   

     Unnamed: 4    Unnamed: 5    Unnamed: 6  
0          язык    количество  средн. знач.  
1  тестирования   зачисленных      тестовых  
2           NaN  абитуриентов        баллов  
3           NaN           NaN           NaN  
4    Кыргызский          2877         145.1  
5       Русский          1248         156.7  
6      В целом:          4125         1

In [29]:
Nat[0]
#we have lift off!!!!!



,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,27500,114.0,29.2,0.91,Кыргызский,2877,145.1
5,Русский,17520,130.7,33.4,0.93,Русский,1248,156.7
6,В целом:,45020,120.5,32.0,0.93,В целом:,4125,148.6


In [30]:
#ok! Let's drop "standardnoe", "unnamed 3, and "unnamed 4".
Nat[0].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
Nat[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,количество,средн. знач.
1,тестирования,участников,тестовых,зачисленных,тестовых
2,NaN,тестирования,баллов,абитуриентов,баллов
3,NaN,NaN,NaN,NaN,NaN
4,Кыргызский,27500,114.0,2877,145.1
5,Русский,17520,130.7,1248,156.7
6,В целом:,45020,120.5,4125,148.6


In [31]:
#looking good. Now we want to rename columns
Nat[0].rename(columns={
    'Unnamed: 0': 'language',
    'Unnamed: 1': 'Total_quantity',
    'Unnamed: 2': 'Avg_score',
    'Unnamed: 5': 'Award_quantity',
    'Unnamed: 6': 'Award_average'
}, inplace=True)
Nat[0]


,language,Total_quantity,Avg_score,Award_quantity,Award_average
0,язык,количество,средн. знач.,количество,средн. знач.
1,тестирования,участников,тестовых,зачисленных,тестовых
2,NaN,тестирования,баллов,абитуриентов,баллов
3,NaN,NaN,NaN,NaN,NaN
4,Кыргызский,27500,114.0,2877,145.1
5,Русский,17520,130.7,1248,156.7
6,В целом:,45020,120.5,4125,148.6


In [32]:
Nat[0]['country'] = 'kyrgyzstan'
Nat[0]['region'] = 'kyrgyzstan'
Nat[0]
#excellent! That's one table almost done. Country will be what we merge on. The region bit will be unique to each region. This is the summary data, so it includes ALL of kyrgyzstan. for now let's keep it as is
#one final piece is to drop the indexes 0, 1, 2 and 3. We only have needed data in index 4-6.


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
0,язык,количество,средн. знач.,количество,средн. знач.,kyrgyzstan,kyrgyzstan
1,тестирования,участников,тестовых,зачисленных,тестовых,kyrgyzstan,kyrgyzstan
2,NaN,тестирования,баллов,абитуриентов,баллов,kyrgyzstan,kyrgyzstan
3,NaN,NaN,NaN,NaN,NaN,kyrgyzstan,kyrgyzstan
4,Кыргызский,27500,114.0,2877,145.1,kyrgyzstan,kyrgyzstan
5,Русский,17520,130.7,1248,156.7,kyrgyzstan,kyrgyzstan
6,В целом:,45020,120.5,4125,148.6,kyrgyzstan,kyrgyzstan


In [33]:
Nat[0].drop(index=[0,1,2,3], inplace=True)  # Drops rows
Nat[0]
#hooray! Managed to take one table from pdf and finally get this into a useable format. This needs to be repeated a bunch of times for each of the "nat [#], then merged I suppose
#showing as error, will clear when I rerun, all good below!

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,27500,114.0,2877,145.1,kyrgyzstan,kyrgyzstan
5,Русский,17520,130.7,1248,156.7,kyrgyzstan,kyrgyzstan
6,В целом:,45020,120.5,4125,148.6,kyrgyzstan,kyrgyzstan


In [34]:
NSTTOTAL = pd.DataFrame(Nat[0])
NSTTOTAL
#and its a full dataframe!!! one down, every region left to go.

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,27500,114.0,2877,145.1,kyrgyzstan,kyrgyzstan
5,Русский,17520,130.7,1248,156.7,kyrgyzstan,kyrgyzstan
6,В целом:,45020,120.5,4125,148.6,kyrgyzstan,kyrgyzstan


Ok! So I can load a pdf page using tabula, and have found a mostly simple solution to rebuild out the identified "list" tables from tabula into workeable dataframes. I'll need to cross reference each table to the pdf to ensure it extracts properly, but we have the first working model above, and a plan for merging it all together into one final dataset :-)! talk about painful.






In [35]:
Nat[1]
#in reviewing, this matches the data for Batken. Let's repeat all our prior operations


,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,2830,117.2,30.5,0.91,Кыргызский,409,145.8
5,Русский,440,120.5,31.8,0.93,Русский,40,148.6
6,В целом:,3270,117.6,30.7,0.93,В целом:,449,146.0


In [36]:
Nat[1].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
#looking good. Now we want to rename columns
Nat[1].rename(columns={
    'Unnamed: 0': 'language',
    'Unnamed: 1': 'Total_quantity',
    'Unnamed: 2': 'Avg_score',
    'Unnamed: 5': 'Award_quantity',
    'Unnamed: 6': 'Award_average'
}, inplace=True)
Nat[1].drop(index=[0,1,2,3], inplace=True)  # Drops rows
Nat[1]['country'] = 'kyrgyzstan'
Nat[1]['region'] = 'Batken'
Nat[1]#check We got it! let's take the process forward and make this a dataframe with the region name, then we can do one massive merge operation


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,2830,117.2,409,145.8,kyrgyzstan,Batken
5,Русский,440,120.5,40,148.6,kyrgyzstan,Batken
6,В целом:,3270,117.6,449,146.0,kyrgyzstan,Batken


In [37]:
Batken = pd.DataFrame(Nat[1])
Batken#done. Next one!

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,2830,117.2,409,145.8,kyrgyzstan,Batken
5,Русский,440,120.5,40,148.6,kyrgyzstan,Batken
6,В целом:,3270,117.6,449,146.0,kyrgyzstan,Batken


In [38]:
Nat[2]# this aligns with Bishkek!

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,3399,118.7,31.1,0.91,Кыргызский,245,149.5
5,Русский,7325,141.3,33.3,0.93,Русский,492,164.6
6,В целом:,10724,134.2,34.3,0.93,В целом:,737,159.5


In [39]:
Nat[2].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
#looking good. Now we want to rename columns
Nat[2].rename(columns={
    'Unnamed: 0': 'language',
    'Unnamed: 1': 'Total_quantity',
    'Unnamed: 2': 'Avg_score',
    'Unnamed: 5': 'Award_quantity',
    'Unnamed: 6': 'Award_average'
}, inplace=True)
Nat[2].drop(index=[0,1,2,3], inplace=True)  # Drops rows
Nat[2]['country'] = 'kyrgyzstan'
Nat[2]['region'] = 'Bishkek'
Bishkek = pd.DataFrame(Nat[2])



In [40]:
Bishkek#check and done!

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,3399,118.7,245,149.5,kyrgyzstan,Bishkek
5,Русский,7325,141.3,492,164.6,kyrgyzstan,Bishkek
6,В целом:,10724,134.2,737,159.5,kyrgyzstan,Bishkek


In [41]:
Nat[3]#this looks like Osh City. rinse and repeat.

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,1032,122.3,33.6,0.91,Кыргызский,129,156.2
5,Русский,1208,130.0,34.1,0.93,Русский,94,162.7
6,В целом:,2240,126.4,34.0,0.93,В целом:,223,158.9


In [42]:
Nat[3].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
#looking good. Now we want to rename columns
Nat[3].rename(columns={
    'Unnamed: 0': 'language',
    'Unnamed: 1': 'Total_quantity',
    'Unnamed: 2': 'Avg_score',
    'Unnamed: 5': 'Award_quantity',
    'Unnamed: 6': 'Award_average'
}, inplace=True)
Nat[3].drop(index=[0,1,2,3], inplace=True)  # Drops rows
Nat[3]['country'] = 'kyrgyzstan'
Nat[3]['region'] = 'Osh_city'
Osh_city = pd.DataFrame(Nat[3])
Osh_city

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,1032,122.3,129,156.2,kyrgyzstan,Osh_city
5,Русский,1208,130.0,94,162.7,kyrgyzstan,Osh_city
6,В целом:,2240,126.4,223,158.9,kyrgyzstan,Osh_city


In [43]:
Nat[4]
#comparing back to the pdf, this is the scores for Jalal-Abad

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,4744,112.3,27.1,0.91,Кыргызский,434,143.2
5,Русский,1214,123.0,31.7,0.93,Русский,91,155.5
6,В целом:,5958,114.4,28.5,0.93,В целом:,525,145.3


In [44]:
Nat[4].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
#looking good. Now we want to rename columns
Nat[4].rename(columns={
    'Unnamed: 0': 'language',
    'Unnamed: 1': 'Total_quantity',
    'Unnamed: 2': 'Avg_score',
    'Unnamed: 5': 'Award_quantity',
    'Unnamed: 6': 'Award_average'
}, inplace=True)
Nat[4].drop(index=[0,1,2,3], inplace=True)  # Drops rows
Nat[4]['country'] = 'kyrgyzstan'
Nat[4]['region'] = 'Jalal_Abad'
Jalal_Abad = pd.DataFrame(Nat[4])
Jalal_Abad

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,4744,112.3,434,143.2,kyrgyzstan,Jalal_Abad
5,Русский,1214,123.0,91,155.5,kyrgyzstan,Jalal_Abad
6,В целом:,5958,114.4,525,145.3,kyrgyzstan,Jalal_Abad


##break
Ok. I was doing these one by one and that makes zero sense from a programming perspective. There has to be a way to do this all at once. Let's start by identifying the remaining region to tables

In [45]:
Nat[5]
#this is Issyk-Kul


,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,3070,111.7,28.0,0.91,Кыргызский,314,141.2
5,Русский,1656,124.4,31.9,0.93,Русский,143,147.4
6,В целом:,4726,116.2,30.0,0.93,В целом:,457,143.2


In [46]:
Nat[6]
#this is Naryn

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,2577,115.8,29.1,0.91,Кыргызский,369,143.8
5,Русский,790,119.4,31.6,0.93,Русский,75,142.6
6,В целом:,3367,116.6,29.7,0.93,В целом:,444,143.6


In [47]:
Nat[7]
#this is Osh

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,5598,111.9,29.2,0.91,Кыргызский,683,146.0
5,Русский,709,115.2,30.2,0.93,Русский,63,154.7
6,В целом:,6307,112.3,29.3,0.93,В целом:,746,146.7


In [48]:
Nat[8]
#Talas!

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,1551,120.2,29.3,0.91,Кыргызский,161,146.9
5,Русский,343,125.8,29.4,0.93,Русский,37,151.2
6,В целом:,1894,121.2,29.4,0.93,В целом:,198,147.7


In [49]:
Nat[9]
#last but not least, Chui

,Unnamed: 0,Unnamed: 1,Unnamed: 2,стандартноe,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,язык,количество,средн. знач.,отклонение,коэффициент,язык,количество,средн. знач.
1,тестирования,участников,тестовых,тестовых,надежности,тестирования,зачисленных,тестовых
2,NaN,тестирования,баллов,NaN,Альфа Кронбаха,NaN,абитуриентов,баллов
3,NaN,NaN,NaN,баллов,NaN,NaN,NaN,NaN
4,Кыргызский,2699,106.7,25.9,0.91,Кыргызский,133,137.0
5,Русский,3835,122.4,29.8,0.93,Русский,213,150.6
6,В целом:,6534,115.9,29.3,0.93,В целом:,346,145.4


In [53]:
# Iterate through the specified dataframes! What a good idea. Should have listed each 0-9 as a variable from the very start.
for i in [5, 6, 7, 8, 9]:
    # Drop unnecessary columns
    Nat[i].drop(columns=['стандартноe', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

    # Rename columns
    Nat[i].rename(columns={
        'Unnamed: 0': 'language',
        'Unnamed: 1': 'Total_quantity',
        'Unnamed: 2': 'Avg_score',
        'Unnamed: 5': 'Award_quantity',
        'Unnamed: 6': 'Award_average'
    }, inplace=True)

    # Drop the first four rows
    Nat[i].drop(index=[0, 1, 2, 3], inplace=True)

    # Add a new column with country like before
    Nat[i]['country'] = 'kyrgyzstan'


In [54]:
#add each individual region as before
Nat[5]['region'] = 'Issyk-Kul'
Nat[6]['region'] = 'Naryn'
Nat[7]['region'] = 'Osh'
Nat[8]['region'] = 'Talas'
Nat[9]['region'] = 'Chui'



In [56]:
#now let's just make each of those a dataframe!
Issyk_Kul = pd.DataFrame(Nat[5])
Naryn = pd.DataFrame(Nat[6])
Osh = pd.DataFrame(Nat[7])
Talas = pd.DataFrame(Nat[8])
Chui = pd.DataFrame(Nat[9])




In [57]:
#lets test em
Issyk_Kul
Naryn
Osh
Talas
Chui

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,3070,111.7,314,141.2,kyrgyzstan,Issyk-Kul
5,Русский,1656,124.4,143,147.4,kyrgyzstan,Issyk-Kul
6,В целом:,4726,116.2,457,143.2,kyrgyzstan,Issyk-Kul


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,2577,115.8,369,143.8,kyrgyzstan,Naryn
5,Русский,790,119.4,75,142.6,kyrgyzstan,Naryn
6,В целом:,3367,116.6,444,143.6,kyrgyzstan,Naryn


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,5598,111.9,683,146.0,kyrgyzstan,Osh
5,Русский,709,115.2,63,154.7,kyrgyzstan,Osh
6,В целом:,6307,112.3,746,146.7,kyrgyzstan,Osh


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,1551,120.2,161,146.9,kyrgyzstan,Talas
5,Русский,343,125.8,37,151.2,kyrgyzstan,Talas
6,В целом:,1894,121.2,198,147.7,kyrgyzstan,Talas


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,2699,106.7,133,137.0,kyrgyzstan,Chui
5,Русский,3835,122.4,213,150.6,kyrgyzstan,Chui
6,В целом:,6534,115.9,346,145.4,kyrgyzstan,Chui


In [63]:
#final step is to merge all of these on country.
#Looking online, pd.merge is one data frame at a time. Since I cleaned these all up, I can change my approach.
#better instead to concate, or just add these all on top of each other
nst = pd.concat([Batken, Bishkek,Osh_city, Jalal_Abad, Issyk_Kul, Naryn, Osh, Talas, Chui])
nst

,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
4,Кыргызский,2830,117.2,409,145.8,kyrgyzstan,Batken
5,Русский,440,120.5,40,148.6,kyrgyzstan,Batken
6,В целом:,3270,117.6,449,146.0,kyrgyzstan,Batken
4,Кыргызский,3399,118.7,245,149.5,kyrgyzstan,Bishkek
5,Русский,7325,141.3,492,164.6,kyrgyzstan,Bishkek
6,В целом:,10724,134.2,737,159.5,kyrgyzstan,Bishkek
4,Кыргызский,1032,122.3,129,156.2,kyrgyzstan,Osh_city
5,Русский,1208,130.0,94,162.7,kyrgyzstan,Osh_city
6,В целом:,2240,126.4,223,158.9,kyrgyzstan,Osh_city
4,Кыргызский,4744,112.3,434,143.2,kyrgyzstan,Jalal_Abad


In [70]:
nst.describe()
#I guess I can drop the country column and the "В целом". Then rename Кыргызский to "Kyrgyz" and "Русский" to Russian.
#However, here's my sixth data set and its almost complete! not the most efficient way, but I figured it out midway how to do this better.


,language,Total_quantity,Avg_score,Award_quantity,Award_average,country,region
count,27,27,27,27,27,27,27
unique,3,27,26,27,25,1,9
top,Кыргызский,2830,112.3,409,146.0,kyrgyzstan,Batken
freq,9,1,2,1,2,27,3
